# 🍩 LIDAR Grasp Detection v14.1

## ✨ แก้ไขใน v14.1
| ปัญหา | วิธีแก้ไข |
|-------|----------|
| Donut grasp ผิด 90° | **แก้ไขมุม** ให้จับแนวรัศมี |
| Gripper เปิดกว้างเกิน | **Pre-open** ให้พอดีความหนาขอบ |
| วัตถุธรรมดา | **ไม่เปลี่ยน** คงวิธีเดิม |

## 1️⃣ Imports

In [ ]:
import cv2
import numpy as np
import time
import socket
import serial

print("✓ Imports (v14.1)")

## 2️⃣ Hardware Configuration

In [ ]:
ROBOT_IP = '192.168.1.6'
ESP32_PORT = 'COM9'
ESP32_BAUDRATE = 115200
CAMERA_ID = 2

HOMOGRAPHY_MATRIX = np.load('homography_matrix.npy')

print("✓ Hardware config")

## 3️⃣ Configuration

In [ ]:
PIXELS_PER_MM = 2.7703
ROBOT_R_OFFSET = -25.55

Z_FLOOR = -64
Z_MEASURE = 120

LIDAR_PHYSICAL_OFFSET = 60
LIDAR_CORRECTION = -21
LIDAR_X_OFFSET = 25.08
LIDAR_Y_OFFSET = 20.71

HEIGHT_CORRECTION_FACTOR = 0.115

DROP_POS = (169.71, 58.01, -17.07, 13.78)

GRIPPER_MAX_WIDTH_MM = 54
GRIPPER_OPEN_MARGIN_MM = 5
GRIPPER_GRIP_MARGIN_MM = 5

MIN_OBJECT_AREA = 800
MAX_OBJECT_AREA = 50000

# Donut Detection Parameters
DONUT_HOLE_RATIO_MIN = 0.1
DONUT_HOLE_RATIO_MAX = 0.7
DONUT_CIRCULARITY_MIN = 0.5

print("✓ Configuration v14.1")

## 4️⃣ SmartGripperController

In [ ]:
class SmartGripperController:
    CALIB_ANGLES = np.array([22, 30, 40, 50, 60, 70, 80, 90, 96])
    CALIB_WIDTHS = np.array([54.0, 52.0, 48.0, 40.0, 32.0, 23.0, 12.0, 3.0, 0.0])
    
    def __init__(self, port='COM9', baudrate=115200):
        self.port = port
        self.baudrate = baudrate
        self.serial = None
        self.target_width = None
        
    def connect(self):
        try:
            self.serial = serial.Serial(self.port, self.baudrate, timeout=2)
            time.sleep(2)
            self.serial.reset_input_buffer()
            print(f"✅ Gripper+LIDAR on {self.port}")
            return True
        except Exception as e:
            print(f"❌ {e}")
            return False
    
    def disconnect(self):
        if self.serial: 
            self.serial.close()
            self.serial = None
    
    def send_command(self, cmd):
        if self.serial and self.serial.is_open:
            self.serial.reset_input_buffer()
            self.serial.write((cmd + '\n').encode())
            time.sleep(0.3)
    
    def mm_to_angle(self, width_mm):
        width = max(0.0, min(54.0, width_mm))
        return int(round(np.interp(width, self.CALIB_WIDTHS[::-1], self.CALIB_ANGLES[::-1])))
    
    def open_for_object(self, width_mm):
        self.target_width = width_mm
        open_w = min(54.0, width_mm + GRIPPER_OPEN_MARGIN_MM)
        angle = self.mm_to_angle(open_w)
        print(f"🦾 Open: {open_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def open_to_width(self, width_mm):
        """v14.1: Open to exact width (for donut pre-positioning)"""
        self.target_width = width_mm
        angle = self.mm_to_angle(width_mm)
        print(f"🦾 Pre-open: {width_mm:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def grip_object(self, width_mm):
        grip_w = max(0.0, width_mm - GRIPPER_GRIP_MARGIN_MM)
        angle = self.mm_to_angle(grip_w)
        print(f"🦾 Grip: {grip_w:.1f}mm ({angle}°)")
        self.send_command(f'G{angle}')
    
    def release(self):
        open_w = min(54.0, (self.target_width or 30) + 10)
        self.send_command(f'G{self.mm_to_angle(open_w)}')
        self.target_width = None
    
    def read_lidar(self, samples=5):
        if not self.serial or not self.serial.is_open:
            return None
        readings = []
        for _ in range(samples):
            self.serial.reset_input_buffer()
            self.serial.write(b'L\n')
            start = time.time()
            while time.time() - start < 1.0:
                if self.serial.in_waiting > 0:
                    response = self.serial.readline().decode().strip()
                    if response.startswith("LIDAR:") and "ERR" not in response:
                        try:
                            readings.append(int(response.split(":")[1]))
                        except:
                            pass
                        break
            time.sleep(0.05)
        return int(np.median(readings)) if readings else None

print("✓ SmartGripperController")

## 5️⃣ DobotControllerTCP

In [ ]:
class DobotControllerTCP:
    def __init__(self, homography_matrix=None, r_offset=-25.55):
        self.dashboard_port = 29999
        self.sock = None
        self.H = homography_matrix
        self.r_offset = r_offset
        
    def connect(self, ip):
        try:
            self.sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            self.sock.settimeout(5)
            self.sock.connect((ip, self.dashboard_port))
            self.send_command("ClearError()")
            time.sleep(0.5)
            self.send_command("EnableRobot()")
            time.sleep(4)
            self.send_command("User(1)")
            self.send_command("Tool(1)")
            self.send_command("SpeedFactor(50)")
            print("✅ Robot connected!")
            return True
        except Exception as e:
            print(f"Error: {e}")
            return False

    def send_command(self, cmd):
        if self.sock:
            self.sock.send((cmd + "\n").encode("utf-8"))
            return self.sock.recv(1024).decode("utf-8")

    def home(self):
        print("🤖 HOME...")
        self.send_command("MovJ(-253.07, 115.17, -17.07, -62.78)")
        time.sleep(4)

    def move_to(self, x, y, z, r=0):
        cmd = f"MovJ({x},{y},{z},{r})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def move_to_and_wait(self, x, y, z, r=0, wait=3):
        self.move_to(x, y, z, r)
        time.sleep(wait)
    
    def joint_move(self, j1=0, j2=0, j3=0, j4=0):
        cmd = f"JointMovJ({j1},{j2},{j3},{j4})"
        print(f"   → {cmd}")
        return self.send_command(cmd)
    
    def joint_move_and_wait(self, j1=0, j2=0, j3=0, j4=0, wait=3):
        self.joint_move(j1, j2, j3, j4)
        time.sleep(wait)

    def pixel_to_robot(self, u, v):
        if self.H is None: return None, None
        pt = np.array([u, v, 1], dtype=np.float32)
        res = np.dot(self.H, pt)
        return res[0]/res[2], res[1]/res[2]
    
    def camera_angle_to_robot_r(self, camera_angle):
        return self.r_offset - camera_angle

print("✓ DobotControllerTCP")

## 6️⃣ ObjectDetectorV14

In [ ]:
class ObjectDetectorV14:
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def detect(self, frame):
        objects = []
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        _, sat_mask = cv2.threshold(s, 50, 255, cv2.THRESH_BINARY)
        _, dark_mask = cv2.threshold(v, 80, 255, cv2.THRESH_BINARY_INV)
        combined_mask = cv2.bitwise_or(sat_mask, dark_mask)
        kernel = np.ones((5, 5), np.uint8)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
        combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
        
        contours, hierarchy = cv2.findContours(
            combined_mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE
        )
        
        if hierarchy is None:
            return objects
        
        hierarchy = hierarchy[0]
        
        for i, cnt in enumerate(contours):
            if hierarchy[i][3] != -1:
                continue
            
            area = cv2.contourArea(cnt)
            if not (MIN_OBJECT_AREA < area < MAX_OBJECT_AREA):
                continue
            
            hole_area = 0
            hole_contour = None
            child_idx = hierarchy[i][2]
            
            while child_idx != -1:
                child_area = cv2.contourArea(contours[child_idx])
                if child_area > hole_area:
                    hole_area = child_area
                    hole_contour = contours[child_idx]
                child_idx = hierarchy[child_idx][0]
            
            hole_ratio = hole_area / area if area > 0 else 0
            perimeter = cv2.arcLength(cnt, True)
            circularity = 4 * np.pi * area / (perimeter * perimeter) if perimeter > 0 else 0
            
            is_donut = (
                DONUT_HOLE_RATIO_MIN <= hole_ratio <= DONUT_HOLE_RATIO_MAX and
                circularity >= DONUT_CIRCULARITY_MIN
            )
            
            hull = cv2.convexHull(cnt)
            rect = cv2.minAreaRect(hull)
            (cx, cy), (w, h), angle = rect
            x, y, bw, bh = cv2.boundingRect(cnt)
            
            objects.append({
                'bbox': (x, y, bw, bh),
                'center': (int(cx), int(cy)),
                'rect': rect,
                'contour': hull,
                'outer_contour': cnt,
                'hole_contour': hole_contour,
                'area': cv2.contourArea(hull),
                'is_donut': is_donut,
                'hole_ratio': hole_ratio
            })
        
        return objects

print("✓ ObjectDetectorV14")

## 7️⃣ DonutGraspSelector v14.1 (Fixed Angle)

In [ ]:
class DonutGraspSelector:
    """v14.1: Fixed grasp angle for donut"""
    
    def __init__(self, pixels_per_mm):
        self.ppm = pixels_per_mm
    
    def analyze_object(self, obj):
        if obj.get('is_donut', False):
            return self._analyze_donut(obj)
        else:
            return self._analyze_solid(obj)
    
    def _analyze_donut(self, obj):
        """v14.1: Fixed - grasp RADIALLY across the ring"""
        outer = obj.get('outer_contour')
        hole = obj.get('hole_contour')
        
        if outer is None or hole is None:
            return self._analyze_solid(obj)
        
        M_outer = cv2.moments(outer)
        M_hole = cv2.moments(hole)
        
        if M_outer['m00'] == 0 or M_hole['m00'] == 0:
            return self._analyze_solid(obj)
        
        cx_outer = int(M_outer['m10'] / M_outer['m00'])
        cy_outer = int(M_outer['m01'] / M_outer['m00'])
        
        outer_rect = cv2.minAreaRect(outer)
        hole_rect = cv2.minAreaRect(hole)
        
        outer_radius = min(outer_rect[1]) / 2
        hole_radius = min(hole_rect[1]) / 2
        ring_thickness_px = outer_radius - hole_radius
        ring_thickness_mm = ring_thickness_px / self.ppm
        
        if ring_thickness_mm <= 0 or ring_thickness_mm > GRIPPER_MAX_WIDTH_MM:
            return self._analyze_solid(obj)
        
        grasps = []
        grasp_radius = (outer_radius + hole_radius) / 2
        
        # 4 positions around the ring
        # angle_deg: 0=Right, 90=Down, 180=Left, 270=Up (in image coordinates)
        for i, angle_deg in enumerate([0, 90, 180, 270]):
            angle_rad = np.radians(angle_deg)
            
            # Grasp point on ring center
            gx = int(cx_outer + grasp_radius * np.cos(angle_rad))
            gy = int(cy_outer + grasp_radius * np.sin(angle_rad))
            
            # v14.1 FIX: Grasp angle should be RADIAL (pointing toward center)
            # This means gripper opens across the ring width
            # angle_deg points outward, so gripper aligns with this direction
            grasp_angle = self._normalize(angle_deg)
            
            # LIDAR point: on the ring (slightly toward outer edge)
            lidar_offset = ring_thickness_px * 0.3
            lx = int(cx_outer + (grasp_radius + lidar_offset) * np.cos(angle_rad))
            ly = int(cy_outer + (grasp_radius + lidar_offset) * np.sin(angle_rad))
            
            grasps.append({
                'center': (gx, gy),
                'lidar_point': (lx, ly),
                'width_mm': ring_thickness_mm,
                'camera_angle': grasp_angle,
                'score': 1.0 - i * 0.1,
                'type': 'Donut-Edge',
                'position': ['Right', 'Bottom', 'Left', 'Top'][i],
                'is_donut_grasp': True  # v14.1: Flag for special handling
            })
        
        return grasps
    
    def _analyze_solid(self, obj):
        """Analyze solid object (same as v13 PCA)"""
        cnt = obj.get('contour')
        if cnt is None or len(cnt) < 5:
            return self._fallback(obj)
        
        pts = cnt.reshape(-1, 2).astype(np.float64)
        mean = np.mean(pts, axis=0)
        pts_centered = pts - mean
        
        cov = np.cov(pts_centered.T)
        eigenvalues, eigenvectors = np.linalg.eig(cov)
        idx = np.argsort(eigenvalues)[::-1]
        eigenvectors = eigenvectors[:, idx]
        
        major = eigenvectors[:, 0]
        minor = eigenvectors[:, 1]
        
        angle = np.degrees(np.arctan2(major[1], major[0]))
        proj = np.dot(pts_centered, minor)
        width_mm = (np.max(proj) - np.min(proj)) / self.ppm
        
        cx, cy = int(mean[0]), int(mean[1])
        grasp_angle = self._normalize(angle + 90)
        
        if width_mm <= GRIPPER_MAX_WIDTH_MM:
            return [{
                'center': (cx, cy),
                'lidar_point': (cx, cy),
                'width_mm': width_mm,
                'camera_angle': grasp_angle,
                'score': 1.0,
                'type': 'PCA-Solid',
                'is_donut_grasp': False
            }]
        return self._fallback(obj)
    
    def _fallback(self, obj):
        rect = obj.get('rect')
        if not rect: return []
        (cx, cy), (w, h), angle = rect
        grip_w = min(w, h) / self.ppm
        grip_a = angle + 90 if w < h else angle
        if grip_w <= GRIPPER_MAX_WIDTH_MM:
            return [{
                'center': (int(cx), int(cy)),
                'lidar_point': (int(cx), int(cy)),
                'width_mm': grip_w,
                'camera_angle': self._normalize(grip_a),
                'score': 0.6,
                'type': 'Rect-Fallback',
                'is_donut_grasp': False
            }]
        return []
    
    def _normalize(self, a):
        while a > 90: a -= 180
        while a < -90: a += 180
        return a

print("✓ DonutGraspSelector v14.1")

## 8️⃣ Initialize Components

In [ ]:
gripper = SmartGripperController(port=ESP32_PORT, baudrate=ESP32_BAUDRATE)
robot = DobotControllerTCP(homography_matrix=HOMOGRAPHY_MATRIX, r_offset=ROBOT_R_OFFSET)
detector = ObjectDetectorV14(PIXELS_PER_MM)
grasp_selector = DonutGraspSelector(PIXELS_PER_MM)
print("✓ Components initialized (v14.1)")

---
# 🤖 CONNECT
---

In [ ]:
print("="*60)
print("🤖 Connecting...")
print("="*60)

gripper.connect()
robot.connect(ROBOT_IP)
robot.home()

print("✅ Ready!")

---
# 🍩 MAIN v14.1
---

In [ ]:
selected_object = None
selected_grasp = None
current_grasps = []
detected_objects = []
grasp_index = 0

def mouse_callback(event, x, y, flags, param):
    global selected_object, selected_grasp, current_grasps, grasp_index
    
    if event == cv2.EVENT_LBUTTONDOWN:
        for obj in detected_objects:
            bx, by, bw, bh = obj['bbox']
            if bx <= x <= bx+bw and by <= y <= by+bh:
                selected_object = obj
                current_grasps = grasp_selector.analyze_object(obj)
                grasp_index = 0
                selected_grasp = current_grasps[0] if current_grasps else None
                if selected_grasp:
                    obj_type = '🍩 DONUT' if obj.get('is_donut') else '⬜ SOLID'
                    print(f"\n{obj_type} Object")
                    print(f"   Type: {selected_grasp['type']}")
                    print(f"   W: {selected_grasp['width_mm']:.1f}mm")
                    print(f"   Angle: {selected_grasp['camera_angle']:.1f}°")
                    if obj.get('is_donut'):
                        print(f"   Position: {selected_grasp.get('position', 'N/A')}")
                break

def draw_frame(frame):
    display = frame.copy()
    
    for obj in detected_objects:
        x, y, w, h = obj['bbox']
        is_sel = (selected_object and obj['center'] == selected_object['center'])
        is_donut = obj.get('is_donut', False)
        
        if is_sel:
            color = (0, 0, 255)
        elif is_donut:
            color = (255, 255, 0)
        else:
            color = (0, 255, 0)
        
        cv2.rectangle(display, (x, y), (x+w, y+h), color, 2)
        
        if is_donut:
            hole_pct = obj.get('hole_ratio', 0) * 100
            cv2.putText(display, f'DONUT {hole_pct:.0f}%', (x, y-5), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, color, 1)
            if obj.get('hole_contour') is not None:
                cv2.drawContours(display, [obj['hole_contour']], -1, (0, 255, 255), 1)
    
    if selected_object and current_grasps:
        for i, g in enumerate(current_grasps):
            cx, cy = g['center']
            angle = g['camera_angle']
            is_selected = (i == grasp_index)
            
            color = (0, 0, 255) if is_selected else (128, 128, 128)
            thick = 3 if is_selected else 1
            
            length = 30
            dx = int(length * np.cos(np.radians(angle)))
            dy = int(length * np.sin(np.radians(angle)))
            cv2.line(display, (cx-dx, cy-dy), (cx+dx, cy+dy), color, thick)
            cv2.circle(display, (cx, cy), 4, color, -1)
            
            lx, ly = g.get('lidar_point', (cx, cy))
            if (lx, ly) != (cx, cy):
                cv2.drawMarker(display, (lx, ly), (255, 0, 255), cv2.MARKER_CROSS, 10, 2)
    
    cv2.rectangle(display, (0, 0), (640, 35), (30, 30, 30), -1)
    cv2.putText(display, f"v14.1 | Obj:{len(detected_objects)} | N=Next | SPACE=Pick",
               (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
    
    if selected_grasp:
        info = f"[{selected_grasp['type']}] W={selected_grasp['width_mm']:.1f}mm A={selected_grasp['camera_angle']:.0f}°"
        if 'position' in selected_grasp:
            info += f" @ {selected_grasp['position']}"
        cv2.putText(display, info, (10, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)
    
    return display

In [ ]:
def pick_v14_1(obj, grasp):
    """v14.1: Pick with donut-specific gripper pre-positioning"""
    cx, cy = grasp['center']
    lx, ly = grasp.get('lidar_point', (cx, cy))
    grip_w = grasp['width_mm']
    camera_angle = grasp['camera_angle']
    is_donut = grasp.get('is_donut_grasp', False)
    
    robot_r = ROBOT_R_OFFSET
    gripper_x, gripper_y = robot.pixel_to_robot(cx, cy)
    lidar_x, lidar_y = robot.pixel_to_robot(lx, ly)
    lidar_x += LIDAR_X_OFFSET
    lidar_y += LIDAR_Y_OFFSET
    
    print(f"\n🤖 Pick v14.1: {grasp['type']}")
    print(f"   W: {grip_w:.1f}mm | Angle: {camera_angle:.1f}°")
    print(f"   Gripper: ({gripper_x:.1f}, {gripper_y:.1f})")
    print(f"   LIDAR: ({lidar_x:.1f}, {lidar_y:.1f})")
    print(f"   Mode: {'DONUT (pre-open)' if is_donut else 'SOLID (normal)'}")
    
    # 1. Safe position
    robot.joint_move_and_wait(0, 0, 0, 0, 1)
    
    # 2. v14.1: Different gripper behavior for donut vs solid
    if is_donut:
        # DONUT: Pre-open to ring thickness + margin (not max width)
        pre_open_width = grip_w + GRIPPER_OPEN_MARGIN_MM + 15
        print(f"🍩 Donut mode: Pre-open to {pre_open_width:.1f}mm")
        gripper.open_to_width(pre_open_width)
    else:
        # SOLID: Normal behavior - open max first
        gripper.open_for_object(GRIPPER_MAX_WIDTH_MM)
    time.sleep(2)
    
    # 3. LIDAR measurement
    print(f"📏 LIDAR measure...")
    robot.move_to_and_wait(lidar_x, lidar_y, Z_MEASURE, robot_r, 2)
    time.sleep(2)
    
    lidar_dist = gripper.read_lidar(samples=100)
    if lidar_dist is None:
        print("❌ LIDAR failed!")
        robot.home()
        return
    
    print(f"📏 LIDAR: {lidar_dist}mm")
    
    # 4. Calculate Z
    z_base = Z_MEASURE - lidar_dist + LIDAR_PHYSICAL_OFFSET + LIDAR_CORRECTION
    est_h = max(0, Z_FLOOR - z_base + (Z_MEASURE - Z_FLOOR))
    z_grasp = z_base - est_h * HEIGHT_CORRECTION_FACTOR
    z_grasp = max(Z_FLOOR, z_grasp)
    print(f"📍 Z_GRASP = {z_grasp:.1f}")
    
    # 5. Move to grasp position
    robot.move_to_and_wait(gripper_x, gripper_y, Z_MEASURE, robot_r, 2)
    final_r = robot.camera_angle_to_robot_r(camera_angle)
    print(f"🔄 Rotating to R={final_r:.1f}°")
    robot.move_to_and_wait(gripper_x, gripper_y, Z_MEASURE, final_r, 2)
    
    # 6. Lower to grasp
    print(f"⬇️ Going down to Z={z_grasp:.1f}")
    robot.move_to_and_wait(gripper_x, gripper_y, z_grasp, final_r, 2)
    
    # 7. Grip
    gripper.grip_object(grip_w - 8.5)
    time.sleep(4)
    
    # 8. Lift & drop
    robot.move_to_and_wait(gripper_x, gripper_y, z_grasp + 50, final_r, 2)
    robot.move_to_and_wait(*DROP_POS[:3], DROP_POS[3], 3)
    gripper.release()
    time.sleep(2)
    
    # 9. Home
    robot.move_to_and_wait(DROP_POS[0], DROP_POS[1], 150, DROP_POS[3], 2)
    robot.joint_move_and_wait(0, 0, 0, 0, 3)
    robot.home()
    print("✅ Complete!")

In [ ]:
# Main loop
cap = cv2.VideoCapture(CAMERA_ID)
cv2.namedWindow('v14.1 Donut')
cv2.setMouseCallback('v14.1 Donut', mouse_callback)

print("="*60)
print("🍩 v14.1 DONUT DETECTION (Fixed Angle + Pre-open)")
print("Click=Select | N=Next | SPACE=Pick | H=Home | R=Reset | Q=Quit")
print("="*60)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret: break
    
    detected_objects = detector.detect(frame)
    display = draw_frame(frame)
    cv2.imshow('v14.1 Donut', display)
    
    key = cv2.waitKey(1) & 0xFF
    
    if key == ord('q'):
        break
    elif key == ord('n') and current_grasps:
        grasp_index = (grasp_index + 1) % len(current_grasps)
        selected_grasp = current_grasps[grasp_index]
        print(f"\n🔄 Grasp {grasp_index+1}/{len(current_grasps)}: {selected_grasp.get('position', selected_grasp['type'])}")
    elif key == ord('r'):
        selected_object = None
        selected_grasp = None
        current_grasps = []
        grasp_index = 0
    elif key == ord('h'):
        robot.home()
    elif key == ord(' ') and selected_object and selected_grasp:
        pick_v14_1(selected_object, selected_grasp)
        selected_object = None
        selected_grasp = None
        current_grasps = []
        grasp_index = 0

cap.release()
cv2.destroyAllWindows()
gripper.disconnect()